In [69]:
import pandas as pd


In [70]:
df = pd.read_csv(r"../data/spam.csv", encoding = "ISO-8859-1")
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


удаляю лишние столбцы

In [71]:
df.drop (df.columns [[2, 3, 4]], axis= 1 , inplace= True )
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


кодирую метки классов

In [72]:
a={"ham": 0,
   "spam": 1}
df["v1"]= df['v1'].map(a)
df

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [73]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer


nlp = spacy.load("en_core_web_sm")


def lemmatize(string):
    return " ".join([word.lemma_ for word in nlp(string)])


df["v2"] = df["v2"].apply(lemmatize)


tf_vectorizer = CountVectorizer(max_df=.95, min_df=10,
                                max_features=20,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(df['v2'])


print(tf.shape)

(5572, 20)


In [74]:
lemmatize("Running and writing")

'run and write'

In [75]:
tf_ar = tf_vectorizer.fit_transform(df['v2']).A
tf_df = pd.DataFrame(tf_ar, columns=tf_vectorizer.get_feature_names_out())
tf_df

,come,day,free,good,gt,just,know,like,love,lt,need,ok,say,send,tell,text,think,time,ur,want
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5568,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5570,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [76]:
target=df["v1"]
target

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: v1, Length: 5572, dtype: int64

In [78]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [79]:
from sklearn.decomposition import PCA
# унижаю размерность предикторов
pca = PCA(n_components=5)
X_pca = pca.fit_transform(tf_df, target)

print(tf_df.shape)
print(X_pca.shape)

(5572, 20)
(5572, 5)


In [80]:
# *** ИСПОЛЬЗОВАТЬ САМОСТОЯТЕЛЬНО РЕАЛИЗОВАННЫЕ МОДЕЛИ ИЗ ПРЕДЫДУЩИХ ЛР
from math import sqrt
from scipy.stats import mode
import numpy as np

class KNNMethod:
    def __init__(self, x_train, y_train, k_neighbors):
        self.x_train=x_train
        self.y_train=y_train.array
        self.k_neighbors = k_neighbors

    def predict(self, x_test):
        x_test=x_test
        classes=[]
        for test_index, test_predictors in enumerate(x_test):
            distances  = []
            for train_index, train_predictors in enumerate(self.x_train):
                distance=0
                for i in range(len(test_predictors)):
                    distance += (test_predictors[i] - train_predictors[i]) ** 2
                distances.append(
                    [sqrt(distance), self.y_train[train_index]]
                )
            distances.sort(key=lambda x: x[0])
            neighbours = distances[:self.k_neighbors]
            classes.append(mode(np.array(neighbours)[:, 1]).mode)
        return classes

In [81]:
#провожу классификацию с помощью моего кнн

X_train, X_test, y_train, y_test = train_test_split(X_pca, target, test_size=0.33)
bag = KNNMethod(X_train, y_train, 9)
print(classification_report(y_test, bag.predict(X_test)))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93      1571
           1       0.83      0.21      0.34       268

    accuracy                           0.88      1839
   macro avg       0.85      0.60      0.64      1839
weighted avg       0.87      0.88      0.85      1839



In [82]:
#провожу классификацию с помощью бэггинга
X_train, X_test, y_train, y_test = train_test_split(X_pca, target, test_size=0.33)
bag = BaggingClassifier().fit(X_train, y_train)
print(classification_report(y_test, bag.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      1590
           1       0.76      0.38      0.51       249

    accuracy                           0.90      1839
   macro avg       0.84      0.68      0.72      1839
weighted avg       0.89      0.90      0.88      1839



## LDA

In [83]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [84]:
n_samples = 2000
n_features = 1000
n_topics = 10
n_top_words = 20

In [85]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [97]:
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=20,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0).fit(tf_df)

In [98]:
# матрица темы-слова
lda.components_.shape

(10, 20)

In [100]:
# матрица документы-темы
lda.transform(tf_df).shape

(5572, 10)

In [101]:
print_top_words(lda, tf_vectorizer.get_feature_names_out(), 9)

Topic #0:
free think just text want ur love like send
Topic #1:
ur day send say good love just tell like
Topic #2:
send say just text like time ok love want
Topic #3:
come ok just day like tell want lt gt
Topic #4:
good time ok love day like want just tell
Topic #5:
text need want free just love tell say like
Topic #6:
gt lt love like send say text need want
Topic #7:
just like want tell text free know love come
Topic #8:
know want like just love say day think tell
Topic #9:
just want like tell love day come ok send
